# OpenMPW bringup notebook

This goal of this notebook is to help you share a reproducible bringup methodology for your [OpenMPW](https://developers.google.com/silicon) project.

- Cells without `%` [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) run on the micropython board.
- Cells with the `%local` magic run on the host computer.

This assumes that the board has already been flashed using the [installation](https://github.com/efabless/caravel_board/tree/main/firmware_vex/nucleo#installation) instructions.

## Connect to the nucleo board

In [55]:
%local
devices=!mpremote connect list | cut -d ' ' -f 1
NUCLEO_DEVICE=devices[0]
NUCLEO_DEVICE

'/dev/ttyACM0'

In [74]:
%serialconnect /dev/ttyACM0
# update the device name to match the device detected


 ** Serial connected **



INFO:micropython-upydevice:Device pyboard connected in /dev/ttyACM0


SerialDevice @ /dev/ttyACM0, Type: pyboard, Class: SerialDevice
Firmware: MicroPython v1.19.1-616-g5987130af-dirty on 2022-11-20; NUCLEO-F746ZG with STM32F746
STM32 STLink, Manufacturer: STMicroelectronics
(MAC: 2f:00:4d:00:02:50:46:56:32:38:36:20)

MicroPython v1.19.1-616-g5987130af-dirty on 2022-11-20; NUCLEO-F746ZG with STM32F746
Type "help()" for more information.


## Run the diagnostic

In [71]:
# update those variable to match the part under test

SHUTTLE='MPW2'  # OpenMPW shuttle ID
SLOT='D5'       # project slot ID
DIE='1'         # actual die ID

PART_NAME=f'{SHUTTLE}_{SLOT}_{DIE}'
print('part name:', PART_NAME)


part name: MPW2_D5_1


In [72]:
import io_config

io_config.run(part_name=PART_NAME, voltage=1.6)


 
io_config -- version 1.2.0
voltage = 1.60, analog = False
 
== Beginning IO configuration test.  Testing LOW IO chain...     ==
 
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_INDEPENDENT >> Failed
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_DEPENDENT   >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_INDEPENDENT >> Failed
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_DEPENDENT   >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_DEPENDENT   >> Passed
gpio[05] - H_INDEPENDENT >> Passed
gpio[06] - H_INDEPENDENT >> Failed
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_DEPENDENT   >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_DEPENDENT   >> Passed
gpio[05] - H_INDEPENDENT >> Passed
gpio[06] - H_DEPENDENT   >> Failed
 
== LOW IO chain test complete.  Testing HIGH IO chain...         ==
 
::::: flashing Caravel :::::
gpio[36] - H_INDEPEN

In [77]:
f = open('gpio_config_def.py')
GPIO_CONFIG = f.read()
print(GPIO_CONFIG)


# gpio_config_def.py file for part MPW2_D5_1
# io_config -- version 1.2.0
voltage = 1.60
analog = False

H_NONE        = 0  
H_DEPENDENT   = 1  
H_INDEPENDENT = 2  
H_SPECIAL     = 3  
H_UNKNOWN     = 4  

# voltage: 1.6
# configuration failed in gpio[6], anything after is invalid
gpio_l = [
['IO[0]', H_NONE],
['IO[1]', H_INDEPENDENT],
['IO[2]', H_DEPENDENT],
['IO[3]', H_INDEPENDENT],
['IO[4]', H_DEPENDENT],
['IO[5]', H_INDEPENDENT],
['IO[6]', H_UNKNOWN],
['IO[7]', H_UNKNOWN],
['IO[8]', H_UNKNOWN],
['IO[9]', H_UNKNOWN],
['IO[10]', H_UNKNOWN],
['IO[11]', H_UNKNOWN],
['IO[12]', H_UNKNOWN],
['IO[13]', H_UNKNOWN],
['IO[14]', H_UNKNOWN],
['IO[15]', H_UNKNOWN],
['IO[16]', H_UNKNOWN],
['IO[17]', H_UNKNOWN],
['IO[18]', H_UNKNOWN],
]
# voltage: 1.6
# configuration failed in gpio[34], anything before is invalid
gpio_h = [
['IO[37]', H_NONE],
['IO[36]', H_DEPENDENT],
['IO[35]', H_INDEPENDENT],
['IO[34]', H_UNKNOWN],
['IO[33]', H_UNKNOWN],
['IO[32]', H_UNKNOWN],
['IO[31]', H_UNKNOWN],
['IO[30]', 

## Run a sanity check

In [107]:
import io_config

io_config.run_sanity_check()


 
io_config -- version 1.2.0
voltage = 1.60, analog = False
== Beginning SANITY for LOW IO chain...                          ==
 
::::: flashing Caravel :::::
gpio[01] - H_INDEPENDENT >> Passed
gpio[02] - H_INDEPENDENT >> Passed
gpio[03] - H_INDEPENDENT >> Passed
gpio[04] - H_INDEPENDENT >> Passed
gpio[05] - H_INDEPENDENT >> Failed
**** SANITY CHECK FOR LOW CHAIN PASSED!!
 
== LOW IO chain test complete.  Testing HIGH IO chain...         ==
 
::::: flashing Caravel :::::
gpio[36] - H_INDEPENDENT >> Passed
gpio[35] - H_INDEPENDENT >> Passed
gpio[34] - H_INDEPENDENT >> Passed
gpio[33] - H_INDEPENDENT >> Passed
gpio[32] - H_INDEPENDENT >> Passed
gpio[31] - H_INDEPENDENT >> Passed
gpio[30] - H_INDEPENDENT >> Passed
gpio[29] - H_INDEPENDENT >> Passed
gpio[28] - H_INDEPENDENT >> Passed
gpio[27] - H_INDEPENDENT >> Passed
gpio[26] - H_INDEPENDENT >> Passed
gpio[25] - H_INDEPENDENT >> Passed
gpio[24] - H_INDEPENDENT >> Passed
gpio[23] - H_INDEPENDENT >> Passed
gpio[22] - H_INDEPENDENT >> Passe

## Run the GPIO test

In [76]:
%local
!rshell -p {NUCLEO_DEVICE} cp gpio_config_def.py ../firmware_vex/gpio_test/
!make -C ../firmware_vex/gpio_test/ gpio_test.hex
!rshell -p {NUCLEO_DEVICE} cp ../firmware_vex/gpio_test/gpio_test.hex /flash/firmware.hex
!echo done

Using buffer-size of 32
Connecting to /dev/ttyACM0 (buffer-size 32)...
Trying to connect to REPL  connected
Retrieving sysname ... pyboard
Testing if ubinascii.unhexlify exists ... Y
Retrieving root directories ... /flash/
Setting time ... Apr 13, 2023 17:48:14
Evaluating board_name ... pyboard
Retrieving time epoch ... Jan 01, 2000
Copying '/home/proppy/src/github.com/efabless/caravel_board/notebooks/gpio_config_def.py' to '/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test/gpio_config_def.py' ...
make: Entering directory '/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test'
python3 ../gpio_config/gpio_config_builder.py
stream_h   = 001111110000001111110000001111110000001111110000001111110000001111110000001111110000001111110000000111110000000011110000000011110000000011110000000011110000000011110000000011110000000001110000000001110000000001110000000000110000000000
stream_l   = 01111111000001111111000001111111000001111111000001111111000001111

In [24]:
import io_config

io_config.run_flash_caravel()


*** flashing Caravel
status Good


## Run project tests

In [77]:
%local
%%writefile ../firmware_vex/gpio_test/gpio_config_io.py
# number of IO in the configuration stream for each chain
NUM_IO = 19

# defines these values for IO configurations
C_MGMT_OUT = 0
C_MGMT_IN = 1
C_USER_BIDIR = 2
C_DISABLE = 3
C_ALL_ONES = 4
C_USER_BIDIR_WPU = 5
C_USER_BIDIR_WPD = 6
C_USER_IN_NOPULL = 7
C_USER_OUT = 8

config_h = [
    C_USER_OUT,  #37
    C_USER_OUT,  #36
    C_USER_OUT,  #35
    C_USER_OUT,  #34
    C_USER_OUT,  #33
    C_USER_OUT,  #32
    C_USER_OUT,  #31
    C_USER_OUT,  #30
    C_USER_OUT,  #29
    C_USER_OUT,  #28
    C_USER_OUT,  #27
    C_USER_OUT,  #26
    C_USER_OUT,  #25
    C_USER_OUT,  #24
    C_USER_OUT,  #23
    C_USER_OUT,  #22
    C_USER_OUT,  #21
    C_USER_OUT,  #20
    C_USER_OUT,  #19
]

del config_h[NUM_IO:]

config_l = [
    C_DISABLE,   #0
    C_DISABLE,   #1
    C_DISABLE,   #2
    C_DISABLE,   #3
    C_DISABLE,   #4
    C_DISABLE,   #5
    C_MGMT_OUT,   #6
    C_DISABLE,   #7
    C_USER_OUT,   #8
    C_USER_OUT,   #9
    C_USER_OUT,   #10
    C_USER_OUT,   #11
    C_USER_OUT,   #12
    C_USER_OUT,   #13
    C_USER_OUT,   #14
    C_USER_OUT,   #15
    C_USER_OUT,   #16
    C_USER_OUT,   #17
    C_USER_OUT,   #18
]

del config_l[NUM_IO:]

Overwriting ../firmware_vex/gpio_test/gpio_config_io.py


In [78]:
%local
%%writefile ../firmware_vex/gpio_test/gpio_test.c

#include "../defs.h"
#include "../gpio_config/gpio_config_io.c"
/*
 * SPDX-FileCopyrightText: 2020 Efabless Corporation
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *      http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 * SPDX-License-Identifier: Apache-2.0
 */


#define BASE_ADDRESS 		0x30000000
#define CTRL_GET_NR		(BASE_ADDRESS + 0x00)
#define CTRL_GET_ID		(BASE_ADDRESS + 0x04)
#define CTRL_SET_IRQ		(BASE_ADDRESS + 0x08)
#define CTRL_FIBONACCI_CTRL	(BASE_ADDRESS + 0x0c)
#define CTRL_FIBONACCI_CLOCK	(BASE_ADDRESS + 0x10)
#define CTRL_FIBONACCI_VAL	(BASE_ADDRESS + 0x14)
#define CTRL_WRITE		(BASE_ADDRESS + 0x18)
#define CTRL_READ		(BASE_ADDRESS + 0x1C)
#define CTRL_PANIC		(BASE_ADDRESS + 0x20)

#define CTRL_ID			0x4669626f
#define CTRL_NR			9

static uint32_t read(unsigned long addr)
{
    return *(volatile uint32_t *)addr;
}

static void write(unsigned long addr, uint32_t val)
{
    *(volatile uint32_t *)addr = val;
}

void set_registers(void)
{
    reg_mprj_io_37 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_36 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_35 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_34 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_33 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_32 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_31 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_30 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_29 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_28 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_27 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_26 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_25 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_24 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_23 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_22 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_21 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_20 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_19 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_18 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_17 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_16 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_15 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_14 = GPIO_MODE_USER_STD_OUTPUT;

    reg_mprj_io_13 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_12 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_11 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_10 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_9 = GPIO_MODE_USER_STD_OUTPUT;
    reg_mprj_io_8 = GPIO_MODE_USER_STD_OUTPUT;
    
    reg_mprj_io_6 = GPIO_MODE_MGMT_STD_OUTPUT;
}

void activate(void)
{
    /* [31:0] is	reg_la0_oenb, 0x2500,0010
     * [63:32] is	reg_la1_oenb, 0x2500,0014
     * [95:64] is	reg_la2_oenb  0x2500,0018
     * [127:96] is	reg_la3_oenb  0x2500,001C
     *
     * All data go on la_data_in[127:0] , which starts
     * at 0x2500,0000
     */
    reg_la1_iena = 0 << 3; /* 0x25000024: Input enable off */
    reg_la1_oenb = 0 << 3; /* 0x25000014: 32th, corresponds to active */
    /* .active() HIGH */
    reg_la1_data = 1 << 3; /* 0x25000004 */
}

void reset(void)
{
    /* .reset(la_data_in[0]) */
    reg_la0_iena = 0;
    reg_la0_oenb = 0; /* 0x2500,0010 */

    reg_la0_data = 1; /* RST on 0x2500,0000*/
    reg_la0_data = 0; /* RST off */
}

#define MAGIC_VAL 0xdeadbeef


void blink_short() {
    const int wait=1000000;
    reg_gpio_mode1 = 1;
    reg_gpio_mode0 = 0;
    reg_gpio_ien = 1;
    reg_gpio_oe = 1;
    reg_gpio_out = 0; delay(wait); // ON
    reg_gpio_out = 1; delay(wait); // OFF
}

void blink_long() {
    const int wait=3000000;
    reg_gpio_mode1 = 1;
    reg_gpio_mode0 = 0;
    reg_gpio_ien = 1;
    reg_gpio_oe = 1;
    reg_gpio_out = 0; delay(wait); // ON
    reg_mprj_datal = 0x00000000;
    reg_mprj_datah = 0x00000000;
    reg_gpio_out = 1; delay(wait); // OFF
    reg_mprj_datal = 0xffffffff;
    reg_mprj_datah = 0xffffffff;    
}

void main()
{
    set_registers();

    gpio_config_io();

    activate();

    // stop fibonacci
    write(CTRL_FIBONACCI_CTRL, 0);
    // update clock
    write(CTRL_FIBONACCI_CLOCK, 1 << 0);
    reset();

    write(CTRL_FIBONACCI_CTRL, 0);
    write(CTRL_FIBONACCI_CLOCK, 1 << 0);
    write(CTRL_FIBONACCI_CTRL, 1);
        
    while (1) {
        blink_short();
    }
}

Overwriting ../firmware_vex/gpio_test/gpio_test.c


In [79]:
%local
!make -C ../firmware_vex/gpio_test/ gpio_test.hex
!rshell -p {NUCLEO_DEVICE} cp ../firmware_vex/gpio_test/gpio_test.hex /flash/firmware.hex
!echo done

make: Entering directory '/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test'
python3 ../gpio_config/gpio_config_builder.py
stream_h   = 001111110000001111110000001111110000001111110000001111110000001111110000001111110000001111110000000111110000000011110000000011110000000011110000000011110000000011110000000011110000000001110000000001110000000001110000000000110000000000
stream_l   = 011111110000011111110000011111110000011111110000011111110000011111110000001111110000001111110000001111110000001111110000000111110000000000000000000111100000000000000000000000000000000000000000000000000000000000000000000000000000000000
n_bits = 247
xpack-riscv-none-elf-gcc-11.3.0-1/bin/riscv-none-elf-gcc -I/home/proppy/src/github.com/efabless/caravel_board/firmware_vex/gpio_test -I../ -I../generated/ -O0 -mabi=ilp32 -march=rv32i -D__vexriscv__ -Wl,-Bstatic,-T,../sections.lds,--strip-debug -ffreestanding -nostdlib -o gpio_test.elf ../crt0_vex.S ../isr.c gpio_test.c
xpack-riscv-none-elf-g

In [82]:
import io_config

io_config.run_flash_caravel()


*** flashing Caravel
status Good


In [100]:
import time
import machine

def sample_fib():
    clk = machine.Pin('TIM3_CH2', machine.Pin.OUT)
    clk.value(0)
    time.sleep_ms(10)
    clk.value(1)
    time.sleep_ms(10)
    n = 0
    for io in range(8, 38):
        n |= machine.Pin(f'IO_{io}', machine.Pin.IN).value() << (io-8)
    return n

buf = [sample_fib() for i in range(100)]
with open('sample_fib.py', 'w') as f:
    f.write(f'BUF = {buf}')
buf


617
[12800, 3904, 262464, 262272, 918016, 393920, 3328, 397248, 400576, 924864, 135552, 3648, 33816576, 33951360, 50863808, 135142720, 168303616, 403050880, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64, 128, 192, 320, 576, 992, 3584, 512, 64, 576, 736, 3328, 4064, 3328, 7424, 512, 3904, 262464, 262272, 918016, 393920, 3328, 397248, 400576, 924864, 131456, 3648, 33816576, 33951360, 50863808, 135142720, 185080832, 403050880, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64, 128, 192, 320, 576, 992, 3584, 512, 64, 576, 736, 3328, 4064, 3328, 7424, 512, 3904, 262464, 262272, 918016, 393920, 3328, 397248, 400576, 924864, 131488, 3648]


In [102]:
%local
!rshell -p {NUCLEO_DEVICE} cp /flash/sample_fib.py .

import pandas as pd
import sample_fib
buf = sample_fib.BUF
start = buf.index(0)

df = pd.DataFrame()

df['samples'] = buf[start:start+44]
df

Using buffer-size of 32
Connecting to /dev/ttyACM0 (buffer-size 32)...
Trying to connect to REPL  connected
Retrieving sysname ... pyboard
Testing if ubinascii.unhexlify exists ... Y
Retrieving root directories ... /flash/
Setting time ... Apr 13, 2023 17:59:01
Evaluating board_name ... pyboard
Retrieving time epoch ... Jan 01, 2000
Copying '/flash/sample_fib.py' to '/home/proppy/src/github.com/efabless/caravel_board/notebooks/sample_fib.py' ...


,samples
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [103]:
%local
import itertools

def fib():
    n = 0
    i = 1
    while True:
        yield n
        n, i = i, i + n

df['expected'] = list(itertools.islice(fib(), 44))
df

,samples,expected
0,0,0
1,0,1
2,0,1
3,0,2
4,0,3
5,0,5
6,0,8
7,0,13
8,0,21
9,0,34


In [104]:
%local
import matplotlib as mpl
import matplotlib.colors as mpl_colors

df_errors = pd.DataFrame()
ios = [f'io[{8+i}]' for i in range(0, 30)]
fibs = [f'fib[{i}]' for i in range(0, 30)]

for i in range(0, 30):
    df_errors[f'io[{8+i}]'] = df['samples'].apply(lambda x: (x >> i) & 1)
    df_errors[f'fib[{i}]'] = df['expected'].apply(lambda x: (x >> i) & 1)

fibs_cmap = mpl_colors.ListedColormap(['honeydew', 'darkseagreen'])
ios_cmap = mpl_colors.ListedColormap(['mistyrose', 'salmon'])

def color_errors(df):
    df_style = df.copy()
    df_style[df[:] ==  0] = 'background-color: honeydew;'
    df_style[df[:] ==  1] = 'background-color: darkseagreen;'
    err_style_0 = 'background-color: mistyrose;'    
    err_style_1 = 'background-color: salmon;'    
    for io, fib in zip(ios, fibs):
        errors = (df[io] != df[fib])
        io_ones = (df[io] == 1)
        fib_ones = (df[fib] == 1)
        df_style.loc[errors & fib_ones, fib] = err_style_1
        df_style.loc[errors & io_ones, io] = err_style_1
        df_style.loc[errors & ~fib_ones, fib] = err_style_0
        df_style.loc[errors & ~io_ones, io] = err_style_0
    return df_style

(df_errors[df_errors.columns[::-1]]
          .style
          .format_index(lambda i: f'fib({i})', axis=0)
          .format_index(lambda c: f'#{c[3:-1]}' if 'io' in c else '=', axis=1)
          .set_table_styles(
            [{'selector': 'td:nth-child(odd)',
              'props': [('border-right', '1px solid black')]},
             {'selector': 'td:nth-child(even)',
              'props': [('border-left', '1px solid black'), ('border-right', '0.5px dashed black')]},
            ]
          )
          .apply(color_errors, axis=None))

,=,#37,=,#36,=,#35,=,#34,=,#33,=,#32,=,#31,=,#30,=,#29,=,#28,=,#27,=,#26,=,#25,=,#24,=,#23,=,#22,=,#21,=,#20,=,#19,=,#18,=,#17,=,#16,=,#15,=,#14,=,#13,=,#12,=,#11,=,#10,=,#9,=,#8
fib(0),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fib(1),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
fib(2),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
fib(3),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
fib(4),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
fib(5),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
fib(6),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
fib(7),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
fib(8),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
fib(9),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
